# **Rodrigo_Mesquita_PB_TP5**
## **Projeto de Bloco: Inteligência Artificial e Machine Learning [24E1_5]**

### 1.Criação das features: Computar o Term Frequency-Inverse Document Frequency (TF-IDF) para representar a importância das palavras em um conjunto de documentos.

In [1]:
#%pip install nltk
#%pip install yellowbrick

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from yellowbrick.cluster import SilhouetteVisualizer
from collections import Counter
from sklearn.metrics import silhouette_score
nltk.download("stopwords")
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
plt.rcParams['font.sans-serif'] = 'DejaVu Sans'

In [4]:
df = pd.read_csv('/workspaces/Faculdade/Datasets/medium.csv')
X = df['Text'].str.lower()

print(f'Quantidade de Documentos: {len(df)}')
df.head(5)

Quantidade de Documentos: 1391


,Title,Text
0,A Beginner’s Guide to Word Embedding with Gens...,1. Introduction of Word2vec\n\nWord2vec is one...
1,Hands-on Graph Neural Networks with PyTorch & ...,"In my last article, I introduced the concept o..."
2,How to Use ggplot2 in Python,Introduction\n\nThanks to its strict implement...
3,Databricks: How to Save Data Frames as CSV Fil...,Photo credit to Mika Baumeister from Unsplash\...
4,A Step-by-Step Implementation of Gradient Desc...,A Step-by-Step Implementation of Gradient Desc...


In [5]:
#Aplicando Regex para manter apenas palavras com 3 ou mais caracteres, retirando pontuações e números
import re
pattern = re.compile(r"\b[a-zA-Z]{3,}\b")
X_reg = X.apply(lambda x: (' '.join(pattern.findall(str(x)))))

In [6]:
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [7]:
#Leematizando o texto
data_lem = []
lemmatizer = WordNetLemmatizer()
for doc in X_reg:
  doc = doc.lower()
  doc_lem = ' '.join(lemmatizer.lemmatize(word) for word in doc.split())
  data_lem.append(doc_lem)

In [14]:
tfidf = TfidfVectorizer(max_df=0.2, min_df=5, stop_words='english')
vect = tfidf.fit_transform(data_lem)

# Convertendo a matriz TF-IDF para um DataFrame
df_tfidf = pd.DataFrame(vect.toarray(), columns= tfidf.get_feature_names_out())

# Exibindo as palavras com maior pontuação:
print(df_tfidf.loc[0].sort_values(ascending=False).head(10))

similarity    0.424331
gensim        0.328443
list          0.262889
embedding     0.204936
cosine        0.188221
sne           0.177180
item          0.173900
vehicle       0.158402
gram          0.150966
van           0.141744
Name: 0, dtype: float64


### 2.Modelagem de Tópicos com LDA: Aplicar o algoritmo LDA para identificar tópicos prevalentes nos dados. A seleção do número de tópicos será baseada em métricas de coerência para garantir a relevância e a distinção entre os tópicos identificados.

In [15]:
#%pip install gensim
#%pip install scipy==1.12

In [16]:
from sklearn.decomposition import LatentDirichletAllocation
from gensim.models import CoherenceModel
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim import corpora, models
import pandas as pd
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [32]:
# Criando o dicionário e o corpus
stop_words = stopwords.words('english')

tokens = [[word for word in doc if word not in stop_words] for doc in [word_tokenize(doc.lower()) for doc in data_lem]]

dicionario = corpora.Dictionary(tokens)

corpus = [dicionario.doc2bow(doc) for doc in tokens]

# Iterar sobre diferentes valores de num_topics
topics_range = [2,3,4,5,6,8,10,12,16,20]
coherence_values = []

for num_topics in topics_range:
    # Construir o modelo LDA
    lda_model = models.LdaMulticore(corpus=corpus, id2word=dicionario,num_topics=num_topics)

    # Calcular a coerência
    coherence_model_lda = CoherenceModel(model=lda_model, texts=tokens, dictionary=dicionario, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    coherence_values.append(coherence_lda)

    print(f"Número de tópicos: {num_topics}, Coerência: {coherence_lda:.4f}")

Número de tópicos: 2, Coerência: 0.2717
Número de tópicos: 3, Coerência: 0.2699
Número de tópicos: 4, Coerência: 0.2730
Número de tópicos: 5, Coerência: 0.2821
Número de tópicos: 6, Coerência: 0.2832
Número de tópicos: 8, Coerência: 0.2731
Número de tópicos: 10, Coerência: 0.2700
Número de tópicos: 12, Coerência: 0.2719
Número de tópicos: 16, Coerência: 0.2757
Número de tópicos: 20, Coerência: 0.2723


In [36]:
stop_words = stopwords.words('english')

tokens = [[word for word in doc if word not in stop_words] for doc in [word_tokenize(doc.lower()) for doc in data_lem]]

dicionario = corpora.Dictionary(tokens)
dicionario.filter_extremes(no_below=5, no_above=0.5)

corpus = [dicionario.doc2bow(doc) for doc in tokens]

# Iterar sobre diferentes valores de num_topics
topics_range = [2, 3, 4, 5, 6, 8, 10, 12, 16, 20]
coherence_values = []

for num_topics in topics_range:
    # Construir o modelo LDA
    lda_model = models.LdaMulticore(corpus=corpus, id2word=dicionario, num_topics=num_topics)

    # Calcular a coerência
    coherence_model_lda = CoherenceModel(model=lda_model, texts=tokens, dictionary=dicionario, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    coherence_values.append(coherence_lda)

    print(f"Número de tópicos: {num_topics}, Coerência: {coherence_lda:.4f}")

Número de tópicos: 2, Coerência: 0.2732
Número de tópicos: 3, Coerência: 0.2779
Número de tópicos: 4, Coerência: 0.2587
Número de tópicos: 5, Coerência: 0.2675
Número de tópicos: 6, Coerência: 0.2683
Número de tópicos: 8, Coerência: 0.2667
Número de tópicos: 10, Coerência: 0.2728
Número de tópicos: 12, Coerência: 0.2620
Número de tópicos: 16, Coerência: 0.2673
Número de tópicos: 20, Coerência: 0.2692


### 3.Classificação de Textos: Desenvolver modelos de classificação para categorizar os textos com base nos tópicos identificados. Você pode escolher qualquer modelo aprendido ao longo do curso e deve escolher o melhor modelo usando as técnicas aprendidas, como busca de hiperparâmetros e validação cruzada

### 4.Avaliação de Desempenho: O desempenho dos modelos de classificação será avaliado utilizando métricas como precisão, recall, F1-score e AUC-ROC.

### 5.Visualização com t-SNE: Aplicar a técnica de t-SNE nos dados textuais vetorizados para reduzir a dimensionalidade e visualizar os agrupamentos de documentos de maneira intuitiva, facilitando a identificação de padrões e outliers.

### 6.Interpretação de Modelos com LIME, SHAP e Force-Plot: Utilizar SHAP para explicar as previsões individuais, identificando a contribuição de cada feature para a decisão do modelo. O force-plot será usado para visualizar essas contribuições de maneira agregada, oferecendo insights sobre a lógica de decisão do modelo.

### 7.Análise dos resultados: Enumere as conclusões que podem ser tomadas a partir dos resultados obtidos.

### 8.Ambiente Recomendado: A solução pode ser desenvolvida em qualquer ambiente de sua escolha, embora seja recomendado o uso do Google Colab Notebook. Lembre-se de comentar seu código para facilitar a compreensão da análise e do procedimento adotado.